In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gt = pd.read_csv("../../Labelling/Lemurs/LemurID_video_eval/groundtruth.csv", sep = ";")
gt.head()

,video,track_id,individual
0,e1_c4_11_19140_20880.txt,1,Red
1,e1_c4_11_19140_20880.txt,6,Her
2,e1_c4_11_19140_20880.txt,8,Gen
3,e1_c4_14_24360_26100.txt,1,Gen
4,e1_c4_14_24360_26100.txt,3,Flo


In [3]:
path_to_validation = "../videos/lemur_id_hyperparams/"
folder_list = sorted([elem for elem in os.listdir(path_to_validation) if not elem.startswith(".")])
folder_list

['additional1_roi_20',
 'additional1_roi_30',
 'additional1_roi_40',
 'additional1_roi_50',
 'base_roi_20',
 'base_roi_30',
 'base_roi_40',
 'base_roi_50',
 'cleaned1_roi_20',
 'cleaned1_roi_30',
 'cleaned1_roi_40',
 'cleaned1_roi_50',
 'cleaned1_squareroi_08_1_20',
 'cleaned1_squareroi_08_1_30',
 'cleaned1_squareroi_08_1_40',
 'cleaned1_squareroi_08_1_50',
 'cleaned1_squareroi_08_1_move20_20',
 'cleaned1_squareroi_08_1_move20_30',
 'cleaned1_squareroi_08_1_move20_40',
 'cleaned1_squareroi_08_1_move20_50',
 'cleaned1_squareroi_20',
 'cleaned1_squareroi_30',
 'cleaned1_squareroi_40',
 'cleaned1_squareroi_50',
 'cleaned1_squareroi_move20_20',
 'cleaned1_squareroi_move20_30',
 'cleaned1_squareroi_move20_40',
 'cleaned1_squareroi_move20_50']

In [7]:

for experiment in folder_list:
    video_list = os.listdir(os.path.join(path_to_validation, experiment))

    prediction_df = pd.DataFrame(columns = ['video', 'track_id', 'prediction', 'track_length'])

    for video_name in video_list:
        
        if video_name.endswith(".txt") and "opt" not in video_name:
            df = pd.read_csv(os.path.join(path_to_validation,experiment,video_name), header=None)

            df.columns = ["frame", "track_id", "V3", "V4", "V5", "V6", "conf", "class", "Cha", "Flo", "Gen", "Geo", "Her", "Rab", "Red", "Uns", "max", "ID"]
            df['max'] = df[['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red']].max(axis=1)

            filtered_df = df[df['class'] == 0]

            # Group by track and calculate various statistics
            result_df = pd.DataFrame()
            result_df['track_id'] = filtered_df['track_id'].unique()

            # Calculate the individual with the highest average probability across the track
            result_df['highest_avg_individual'] = filtered_df.groupby('track_id')[['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red']].mean().idxmax(axis=1).values
            result_df['highest_avg_probability'] = filtered_df.groupby('track_id')[['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red']].mean().max(axis=1).values

            # Calculate the individual with the single highest probability value across the track
            result_df['single_highest_individual'] = filtered_df.groupby('track_id')[['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red']].max().idxmax(axis=1).values
            result_df['single_highest_probability'] = filtered_df.groupby('track_id')[['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red']].max().max(axis=1).values

            # Calculate the individual with the highest number of times getting a probability value over 0.8 within the track
            counts_over_thres = filtered_df[filtered_df['max'] > 0.8].groupby('track_id').apply(lambda x: x[['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red']].idxmax(axis=1).value_counts().reset_index().rename(columns={'index': 'individual', 0: 'count'}) if not x.empty else pd.DataFrame({'individual': ['None'], 'count': [0]}))
            if len(counts_over_thres)>0:
                counts_over_thres = counts_over_thres.reset_index(level=1, drop = True)
                counts_over_thres = counts_over_thres.sort_values(by='count', ascending=False).groupby('track_id').first()

                # Merge with result_df using an outer join
                result_df = pd.merge(result_df, counts_over_thres, on='track_id', how='outer')

            else:
                result_df['count'] = 0
                result_df['individual']= ""

            # Add total length of the track
            result_df['track_length'] = filtered_df.groupby('track_id').size().values

            #print(result_df)

            thresholds = {'highest_avg_probability': 0.1, 'single_highest_probability': 0.8, 'count': 1}

            value_cols = ['highest_avg_probability', 'single_highest_probability', 'count']
            name_cols = ['highest_avg_individual', 'single_highest_individual', 'individual']

            values_df = result_df[value_cols].ge(pd.Series(thresholds))
            names_df=result_df[name_cols].fillna('').rename(columns={'highest_avg_individual': 'highest_avg_probability', 'single_highest_individual': 'single_highest_probability', 'individual': 'count'})

            #median of all 3
            result_df['prediction'] =names_df.where(values_df, "").replace("", "Uns").apply(lambda row: row.mode().iloc[0], axis = 1)

            #only highest 
            result_df['prediction_count'] =names_df.where(values_df, "").replace("", "Uns")['count']
            result_df['prediction_singlehighest'] =names_df.where(values_df, "").replace("", "Uns")['single_highest_probability']
            result_df['prediction_avghighest'] =names_df.where(values_df, "").replace("", "Uns")['highest_avg_probability']
            result_df['video'] = video_name
            #print(result_df[['video', 'track_id', 'final']])
            prediction_df = pd.concat([prediction_df, result_df[['video', 'track_id', 'prediction', 'track_length']].fillna("Uns")], ignore_index=True)
        

    names_df.where(values_df, "").replace("", "Uns")

    final_df = pd.merge(gt, prediction_df, on =['video', 'track_id'], how = "left")
    classes = sorted(set(final_df['individual']).union(final_df['prediction']))
    cha = False
    if "Cha" in classes:
        classes.remove("Cha")
        cha = True

    # Calculate confusion matrix
    conf_matrix = pd.crosstab(final_df['individual'], final_df['prediction'], rownames=['Actual'], colnames=['Predicted'])

    if cha:
        conf_matrix["Uns"] = conf_matrix["Uns"] + conf_matrix["Cha"]

        # Drop the "Cha" column
        conf_matrix.drop(columns=["Cha"], inplace=True)


    missing_classes = set(classes) - set(conf_matrix.columns)
    for missing_class in missing_classes:
        conf_matrix[missing_class] = 0

    # Sort the columns and index to maintain order
    conf_matrix = conf_matrix[sorted(set(conf_matrix.columns))].sort_index()


    accuracy = np.diag(conf_matrix).sum() / conf_matrix.values.sum()
    print(experiment, "\tAccuracy:", accuracy)

additional1_roi_20 	Accuracy: 0.6304347826086957
additional1_roi_30 	Accuracy: 0.6630434782608695
additional1_roi_40 	Accuracy: 0.6086956521739131


Accuracy 0.6304347826086957
